In [12]:
#Chargement des données
# Data
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os 
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import math
# Graphics
import seaborn as sns ; sns.set()

from sklearn.experimental import enable_halving_search_cv # noqa

from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import FunctionTransformer
# from sklearn.metrics import root_mean_squared_error

In [13]:
load_dotenv('BDD_URL.env')
BDD_URL = os.environ['BDD_URL']
engine = create_engine(BDD_URL)

SQL_filtre= """
SET search_path to principal;
SELECT *
from "filmview"
where 'Comedy' = ANY(string_to_array("genres", ','))
limit 10000;
"""
SQL= """
SET search_path to principal;
SELECT *
from "filmview"
where "runtimeMinutes" Is NOT null and "titleType" = 'movie' and "averageRating" is NOT NULL and "genres" is NOT NULL and "startYear" is NOT NULL and "isAdult" is NOT NULL
limit 30000;
"""
df = pd.read_sql(SQL, engine)
engine.dispose()
df

,tconst,primaryTitle,titleType,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,Cate&names
0,tt4534492,In the Dark,movie,0,2015,None,85,"Horror,Thriller",4.3,193,"[composer_Drew_Silverstein, director_Chris_St...."
1,tt4534598,Legend of Destruction,movie,0,2021,None,93,Animation,7.9,244,"[editor_Ori_Ben-Dov, actor_Elliott_Gould, comp..."
2,tt4534732,Maiko: Dancing Child,movie,0,2015,None,70,"Biography,Documentary",6.6,60,"[director_Åse_Svenheim_Drivenes, producer_Tone..."
3,tt4534804,"I, Superbiker 5: Split Second",movie,0,2015,None,90,Documentary,3.7,20,"[director_Mark_Sloper, editor_David_Hughes]"
4,tt4534886,Making Out,movie,0,2016,None,85,"Comedy,Romance",4.9,113,"[actress_Iabou_Windimere, actor_Titus_Young_Wo..."
...,...,...,...,...,...,...,...,...,...,...,...
29995,tt8168050,Teidät tuomitaan elinkautiseen,movie,0,2018,None,85,Drama,4.5,6,"[director_Miika_Abelsson, writer_Antti_Välikan..."
29996,tt8168814,Aashiq Vanna Divasam,movie,0,2018,None,127,"Drama,Family",5.5,12,"[actor_Vj_Stajan, actor_Ansar_Kalabhavan, edit..."
29997,tt8168884,Unholy Night,movie,0,2019,None,85,"Comedy,Horror",4.2,187,"[director_Randy_Smith, director_Kristian_Lariv..."
29998,tt8168944,Havana Libre,movie,0,2021,None,82,Documentary,8.9,9,"[producer_Nicholas_Weissman, cinematographer_S..."


In [14]:
def listTostr (df):
    return df.apply(lambda x: ' '.join(map(str, x)))

In [15]:
# df["averageRating"].fillna(df["averageRating"].mean(), inplace=True)
# df["startYear"].fillna(df["startYear"].mean(), inplace=True)
# df["runtimeMinutes"].fillna(df["runtimeMinutes"].mean(), inplace=True)
# df["isAdult"] = df["isAdult"].apply(lambda x: 1 if x == "True" else 0)

# df['isAdult'] = df['isAdult'].astype(bool)
df['Cate&names'].fillna('missing', inplace=True)
df['Cate&names']=df[['Cate&names']].apply(listTostr)
# df['Cate&names']=df['Cate&names'].str.replace('_', ' ')
df['genres'].fillna('missing', inplace=True)
df['genres']=df['genres'].str.replace(',', ' ')
df


,tconst,primaryTitle,titleType,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,Cate&names
0,tt4534492,In the Dark,movie,0,2015,None,85,Horror Thriller,4.3,193,composer_Drew_Silverstein director_Chris_St.Cr...
1,tt4534598,Legend of Destruction,movie,0,2021,None,93,Animation,7.9,244,editor_Ori_Ben-Dov actor_Elliott_Gould compose...
2,tt4534732,Maiko: Dancing Child,movie,0,2015,None,70,Biography Documentary,6.6,60,director_Åse_Svenheim_Drivenes producer_Tone_G...
3,tt4534804,"I, Superbiker 5: Split Second",movie,0,2015,None,90,Documentary,3.7,20,director_Mark_Sloper editor_David_Hughes
4,tt4534886,Making Out,movie,0,2016,None,85,Comedy Romance,4.9,113,actress_Iabou_Windimere actor_Titus_Young_Wolv...
...,...,...,...,...,...,...,...,...,...,...,...
29995,tt8168050,Teidät tuomitaan elinkautiseen,movie,0,2018,None,85,Drama,4.5,6,director_Miika_Abelsson writer_Antti_Välikanga...
29996,tt8168814,Aashiq Vanna Divasam,movie,0,2018,None,127,Drama Family,5.5,12,actor_Vj_Stajan actor_Ansar_Kalabhavan editor_...
29997,tt8168884,Unholy Night,movie,0,2019,None,85,Comedy Horror,4.2,187,director_Randy_Smith director_Kristian_Larivie...
29998,tt8168944,Havana Libre,movie,0,2021,None,82,Documentary,8.9,9,producer_Nicholas_Weissman cinematographer_Set...


In [16]:
# Séparation des caractéristiques et de la cible
X = df.drop(columns=["averageRating"])
y = df["averageRating"]

# Séparation des données d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)#, random_state=42



In [17]:
# Définition des colonnes numériques, textuelles et de description
numeric_features = ['startYear', 'runtimeMinutes']
boolean_features = 'isAdult'
text_features = 'titleType'
title = 'primaryTitle'#['primaryTitle''titleType', 'directors', 'writers']
genre = 'genres'
description_feature = 'Cate&names'

In [18]:
# Création des transformers pour les colonnes numériques, booléennes, textuelles et de description
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

boolean_transformer = FunctionTransformer(lambda x: x.astype(bool).values.reshape(-1, 1)) 

text_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

Vect_transformer = Pipeline([
    # ('imputer', SimpleImputer(strategy='constant', fill_value='missing')), 
    ('vect', CountVectorizer(decode_error='ignore', analyzer='word')) #max_features=1000, analyzer="word"
])
tfidf_transformer = Pipeline([
    # ('imputer', SimpleImputer(strategy='constant', fill_value='missing')), 
    ('tf_idf', TfidfVectorizer(decode_error='ignore', analyzer='word')) #max_features=1000
])
# Création d'un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('bool', boolean_transformer, boolean_features),
        ('text', Vect_transformer, text_features),
        ('title', tfidf_transformer, title),
        ('genre', tfidf_transformer, genre),
        ('description', tfidf_transformer, description_feature)
    ])
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['startYear', 'runtimeMinutes']),
                                ('bool',
                                 FunctionTransformer(func=<function <lambda> at 0x000002DE5AB404A0>),
                                 'isAdult'),
                                ('text',
                                 Pipeline(steps=[('vect',
                                                  CountVectorizer(decode_error='ignore'))]),
                                 'titleType'),
                                ('title',
                                 Pipeline(steps=[('tf_idf',
                                                  TfidfVectorizer(decode_error='ignore'))]),
                                 'primaryTitle'),
                                ('genre',
                                 Pipeline(steps=[('tf_idf',
                                                  TfidfVectorizer(decode_error='ignore'))]),
                                 'genres'),
                                ('description',
                                 Pipeline(steps=[('tf_idf',
                                                  TfidfVectorizer(decode_error='ignore'))]),
                                 'Cate&names')])

In [19]:
# Création des pipelines pour chaque modèle
pipelines = {
    'Linear Regression': Pipeline([('preprocessor', preprocessor), ('regressor', LinearRegression())]),
    'Ridge Regression': Pipeline([('preprocessor', preprocessor), ('regressor', Ridge())]),
    'Lasso Regression': Pipeline([('preprocessor', preprocessor), ('regressor', Lasso())]),
    'ElasticNet': Pipeline([('preprocessor', preprocessor), ('regressor', ElasticNet())]),
    'SVR': Pipeline([('preprocessor', preprocessor), ('regressor', SVR())]),
    'Random Forest Regression': Pipeline([('preprocessor', preprocessor), ('regressor', RandomForestRegressor())]),
    'Gradient Boosting Regression': Pipeline([('preprocessor', preprocessor), ('regressor', GradientBoostingRegressor())]),
    
}

# Paramètres pour GridSearchCV pour chaque modèle
parameters = {
    'Linear Regression': {'regressor__fit_intercept': [True,False]},
    'Ridge Regression': {'regressor__alpha': [0.1, 5.0, 10.0]},
    'Lasso Regression': {'regressor__alpha': [0.1, 5.0, 10.0]},
    'ElasticNet': {'regressor__alpha': [0.1, 5.0, 10.0], 'regressor__l1_ratio': [0.1, 0.5, 0.9]},
    'SVR': {'regressor__kernel': ['linear', 'rbf'], 'regressor__C': [0.1, 1.0, 10.0]},
    'Random Forest Regression': {'regressor__n_estimators': [150,200], 'regressor__max_depth': [50,100]},# None, 
    'Gradient Boosting Regression': {'regressor__n_estimators': [150,200], 'regressor__max_depth': [50, 100]},
    
}

# Scoring : RMSE, R2 et MAE
scoring = {'RMSE': 'neg_root_mean_squared_error',
           'R2': 'r2',
           'MAE': 'neg_mean_absolute_error'}


In [20]:
def Grid(X_train, y_train, pipeline, parameters, cv=5):
  # Scoring
  #multi_scoring = {mean_squared_error,r2_score}
    
    
  # Grid search
  grid = GridSearchCV(pipeline, parameters,  scoring=scoring, refit='RMSE', cv=cv, n_jobs =-1, verbose = 0, error_score='raise')

  # Fit
  grid.fit(X_train, y_train)

  # Scores and results
  best_score = grid.best_score_.round(4)
  best_params = grid.best_params_
  training_time = grid.cv_results_['mean_fit_time'].mean().round(4)

  # Output
  return({
      'best_score': best_score,
      'best_params': best_params,
      'training_time': training_time,
      'fitted_model': grid.best_estimator_
  })

In [21]:
def afficheResults (grid):
    model_name = grid['fitted_model'].named_steps['regressor'].__class__.__name__
    print(f"{model_name} training time: {grid['training_time']}")
    print(f"Best {model_name} parameters: {grid['best_params']}")
    print(f"Best {model_name} score: {-grid['best_score']}")
 
    
    

In [22]:

# Boucle sur les modèles pour ajuster avec GridSearchCV
models = {}

# Boucle sur les modèles pour ajuster avec GridSearchCV et évaluation
for model_name, pipeline in pipelines.items():
    print(f"\n..............{model_name}..............................")
    grid_search = Grid(X_train, y_train, pipeline, parameters[model_name], cv=5)
    afficheResults (grid_search)
    
    best_model = grid_search['fitted_model']
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    Rmse = round(math.sqrt(mse), 4)
    models[model_name] = [best_model,Rmse]
    print(f"{model_name} RMSE: {Rmse}")
 


..............Linear Regression..............................
LinearRegression training time: 23.828
Best LinearRegression parameters: {'regressor__fit_intercept': False}
Best LinearRegression score: 1.2479
Linear Regression RMSE: 1.2449

..............Ridge Regression..............................
Ridge training time: 5.3471
Best Ridge parameters: {'regressor__alpha': 5.0}
Best Ridge score: 1.2238
Ridge Regression RMSE: 1.2267

..............Lasso Regression..............................


KeyboardInterrupt: 

In [ ]:
'''
# Boucle sur les modèles pour ajuster avec GridSearchCV
results = {}

# Boucle sur les modèles pour ajuster avec GridSearchCV et évaluation
for model_name, pipeline in pipelines.items():
    print(f"Training {model_name}...")
    grid_search = GridSearchCV(pipeline, parameters[model_name],  scoring=scoring, refit='RMSE', cv=5, n_jobs =-1, verbose = 1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    mse = root_mean_squared_error(y_test, y_pred)
    results[model_name] = mse
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best {model_name} score: {-grid_search.best_score_}")
    print(f"{model_name} MSE: {mse}")
    print()
'''

'\n# Boucle sur les modèles pour ajuster avec GridSearchCV\nresults = {}\n\n# Boucle sur les modèles pour ajuster avec GridSearchCV et évaluation\nfor model_name, pipeline in pipelines.items():\n    print(f"Training {model_name}...")\n    grid_search = GridSearchCV(pipeline, parameters[model_name],  scoring=scoring, refit=\'RMSE\', cv=5, n_jobs =-1, verbose = 1)\n    grid_search.fit(X_train, y_train)\n    best_model = grid_search.best_estimator_\n    y_pred = best_model.predict(X_test)\n    mse = root_mean_squared_error(y_test, y_pred)\n    results[model_name] = mse\n    print(f"Best parameters: {grid_search.best_params_}")\n    print(f"Best {model_name} score: {-grid_search.best_score_}")\n    print(f"{model_name} MSE: {mse}")\n    print()\n'

In [ ]:
'''
# Evaluation des modèles sur les données de test
print("\nComparaison des performances des modèles sur les données de test:")
for model_name, mse in results.items():
    print(f"{model_name}: MSE = {mse}")
    
'''

'\n# Evaluation des modèles sur les données de test\nprint("\nComparaison des performances des modèles sur les données de test:")\nfor model_name, mse in results.items():\n    print(f"{model_name}: MSE = {mse}")\n    \n'